In [25]:
#Alright alright alright, let's begin this data cleaning by importing the correct modules and the data itself. 

import pandas as pd
import re
import random

data = pd.read_csv('/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Data Transformation/Projects/project-pandas/GSAF5.csv', encoding = "latin-1")

#Let's also have a quick look at it. 
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [26]:
#As a next step, let's check exaclty what will we be handling from this point on. 

#Dataset is 5,992 rows by 24 columns. 
print(data.shape)
print('\n')

#There are some weird columns which have no aparent meaning for the Shark topic of the day. 
print(data.columns)
print('\n')

#Finally, most of the columns contain text, Jesus Christ...
print(data.dtypes)

(5992, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number

In [27]:
#1. Column Cleaning. 

#First of all, let's check how many empty cells there are in every column. 
null_columns = data.isnull().sum()
print(null_columns[null_columns > 0])

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


In [28]:
#lol, columns Unnamed: 22 and Unnamed: 23 are quite the slackers regarding information storage it seems. Let's check them out in detail and replace NA values with 0s until something pops up. 
data['Unnamed: 22'].fillna(0, inplace = True)
print([x for x in data['Unnamed: 22'] if x != 0])

data['Unnamed: 23'].fillna(0, inplace = True)
print([y for y in data['Unnamed: 23'] if y != 0]) 

['stopped here']
['Teramo', 'change filename']


In [29]:
#These two columns have the above 3 items, so let's just delete them both.
data = data.drop('Unnamed: 22', 1)
data = data.drop('Unnamed: 23', 1)

In [30]:
#So, onto the next 3 suspicious columns: Case Number.2, Case Number, and Case Number .1.

#Let's check if these are duplicates of some sort.
print(data['Case Number.1'].equals(data['Case Number.2'].equals(data['Case Number'])))
print('\n')

#Nope, those are not exact duplicates apparently, now let's see if data from one column can fe found in the other two. 
print(data['Case Number.1'].isin(data['Case Number.2']).value_counts())
print('\n')

#Interesting, only 10 values are different from one column to the other one. Let's do a final check with the last column as well. 
print(data['Case Number'].isin(data['Case Number.1']).value_counts())

False


True     5982
False      10
Name: Case Number.1, dtype: int64


True     5979
False      13
Name: Case Number, dtype: int64


In [31]:
#Same story here, only 13 different values, let's then delete Case Number.1 and Case Number.2
data = data.drop('Case Number.1', 1)
data = data.drop('Case Number.2', 1)

In [32]:
#Finally let's check the Href Formula and Href Columns. 
data['href formula'].isin(data['href']).value_counts()

True     5939
False      53
Name: href formula, dtype: int64

In [33]:
#Well, values from href formula Column that are in href Column are quite more than in our past findings, however these are still not more than 10% of the whole thing, so I'll delete Href Formula Column. Moreover, Href Column looks slimmer and more presentable by itself. 
data = data.drop('href formula', 1)

#Now we have an aparently nice table of stuff, let's check it out. 
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [34]:
#2. Data Cleaning

#From this point on, let's actually clean the data inside each of the columns we left, starting from the Data Column, which is always pivotal in an anaysis. 
set(data['Date'].sample(n=30))

{' 14-Aug-2013',
 '02-Aug-97',
 '03-Mar-1875',
 '05-Mar-96',
 '06-Jan-15',
 '06-May-90',
 '07-Jan-62',
 '07-Mar-07',
 '07-Mar-61',
 '10-Feb-16',
 '12-Nov-05',
 '13-Jan-74',
 '13-May-11',
 '16-Nov-59',
 '1748',
 '1949',
 '20-Aug-06',
 '20-May-01',
 '24-May-14',
 '25-Dec-12',
 '26-Mar-44',
 '27-May-13',
 '28-Dec-97',
 '28-Mar-64',
 '28-Sep-85',
 '30-Nov-86',
 'Apr-13',
 'Before 1871',
 'Ca. 1960',
 'Circa 1855'}

In [38]:
#So yeah, as we can see, format is everywhere from single year values, to literally strings of text saying when did the incident "maybe" happened. 

#To solve this, let's break the problem into quick wins. First let's use the column Case Number to check against the Year Column to check if everything corresponds to at least the correct year. 

#The following code will create 3 Columns to separate the date of an incident, based on the Case Number Column, and check against Date and Year Columns if something is off. 
try:
    data['Year_Check']=[d[0:4] for d in data['Case Number']]
except:
    data['Year_Check']='00'
try:
    data['Month']=[d[5:7] for d in data['Case Number']]
except:
    data['Month']='00'
try:
    data['Day']=[d[8:10] for d in data['Case Number']]
except:
    data['Day']='00'
data.loc[data['Year_Check'].str.contains('(?i)ND'),'Year_Check']='00'
data.loc[data['Year_Check'].str.contains('ND'),'Day']='00'
data.loc[data['Year_Check'].str.contains('ND'),'Month']='00'
data['Year_Check'] = data['Year_Check'].replace(['0.02','0.03','0.04','0.07'],0)
data['Year_Check'] = data['Year_Check'].astype(int)

#Now let's order the dataframe by useful comparing Columns. 
data = data[['Case Number','Day','Month','Year_Check','Year','Date','Type','Country','Area','Location','Activity','Name',
           'Sex ','Age','Injury','Fatal (Y/N)','Species ','Investigator or Source','href']]

data.head()

,Case Number,Day,Month,Year_Check,Year,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,href
0,2016.09.18.c,18,09,2016,2016,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...
1,2016.09.18.b,18,09,2016,2016,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...
2,2016.09.18.a,18,09,2016,2016,18-Sep-16,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...
3,2016.09.17,17,09,2016,2016,17-Sep-16,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...
4,2016.09.15,15,09,2016,2016,16-Sep-16,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...


In [39]:
#Ok let's check if every year from our Check Column coincides with every year from our Year Column.
data.loc[data['Year_Check'] != (data['Year'])]

,Case Number,Day,Month,Year_Check,Year,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Species,Investigator or Source,href
2449,1989.07.27,27,07,1989,1969,27-Jul-89,Invalid,BERMUDA,NaN,NaN,Scuba diving,Russian male,M,35,FATAL,NaN,Shark involvement suspected but not confirmed,"LA Times, 7/28/1989",http://sharkattackfile.net/spreadsheets/pdf_di...
3662,1961.07.16,16,07,1961,1971,16-Jul-61,Unprovoked,TURKEY,Anatolia,"?nciralti Beach, ?zmir",Swimming,?brahim Karagöz,M,16,Left leg injured,N,NaN,"C. Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
4312,1948.09.17.R,17,09,1948,1848,Reported 17-Sep-1848,Unprovoked,TURKEY,Adana Province,Yumurtalik,Swimming,Ali Kaymaz,M,NaN,FATAL,Y,NaN,"C. Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...
4983,1923.00.00.a,00,00,1923,1922,1923,Provoked,USA,New Jersey,Ocean City (offshore),Hoisting shark aboard fishing boat,male,M,NaN,Shark's tail broke his leg. PROVOKED INCIDENT,N,NaN,"Ref in New York Herald Tribune, 8/23/1960; V.M...",http://sharkattackfile.net/spreadsheets/pdf_di...
5043,1900.00.00.R,00,00,1900,1919,Reported to have taken place in 1919,Boating,ITALY,NaN,Savona,Fishing,NaN,M,NaN,No injury,N,13' shark,"C. Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...


In [40]:
#Voila! 5 rows do not completely coincide, moreover, the Date Column will help us decide wich year is the correct one. Let's change that right now. 
data.loc[4983,'Year'] = 1923
data.loc[3662,'Year'] = 1961
data.loc[2449,'Year'] = 1989 

#Also this would be a good time to rename the Date Column to a more understandable Date and Commentary Column. 
data.rename(columns={'Date': 'Date and Commentary'}, inplace = True)

In [41]:
#Now let's check the Fatal Column and see if it has only Y or N values. 
set(data['Fatal (Y/N)'])

{' N', '#VALUE!', 'F', 'N', 'N ', 'UNKNOWN', 'Y', 'n', nan}

In [42]:
#Naturally, it is a mess as well, so let's change all those values to binary responses. 
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.strip()
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].fillna('U')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('n', 'N')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('#VALUE!', 'U')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('F', 'Y')
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].str.replace('UNKNOWN', 'U')

#I will also rename the Column to Fatal Y/N/U, based on the fact that Unknown is also a valid response I think. 
data.rename(columns = {'Fatal (Y/N)': 'Fatal (Y/N/U)'}, inplace = True)
print(set(data['Fatal (Y/N/U)']))

{'N', 'Y', 'U'}
